Una pregunta el codigo que tenemos hasta ahora solo esta diseñado para escanear un pdf o cual es el limite en que podriamos arreglar el codigo para que lee N° de pdf? tambien de que por cada pdf va a ver un json differente y separado del anterior?

In [1]:
import spacy
import re
import fitz  # PyMuPDF
import json
import os

def extraer_metadatos_gaceta_spacy(texts_pdf):
    """
    Extrae metadatos de una gaceta del Congreso usando spaCy y expresiones regulares.
    """
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(texts_pdf)
    
    metadatos = {}
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            metadatos["fecha"] = ent.text
        elif ent.label_ == "CARDINAL" and "páginas" in ent.sent.text.lower():
            metadatos["num_paginas"] = int(ent.text)
        elif ent.label_ == "ORG" and "directores" in ent.sent.text.lower():
            metadatos["directores"] = ent.text
        elif ent.label_ == "MISC" and "ISSN" in ent.sent.text:
            metadatos["issn"] = ent.text.split()[-1]

    # --- 1. Fecha ---
    pattern_date = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    match_date = re.search(pattern_date, texts_pdf)
    if match_date:
        meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                 "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        day_month = int(match_date.group(2))
        month_num = meses[match_date.group(3).lower()]
        anio = int(match_date.group(4))
        metadatos["fecha"] = f"{day_month:02d}/{month_num:02d}/{anio}"

    # --- 2. Directores ---
    pattern_directores = r"(?i)DIRECTORES:\s*(.*?)\s*(?:SECRETARIO|RAMA|EDICI[ÓO]N)"
    match_directores = re.search(pattern_directores, texts_pdf)
    if match_directores:
        directores_texto = match_directores.group(1).strip()
        directores = re.findall(r"([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\s+(SECRETARIO GENERAL DEL SENADO|SECRETARIO GENERAL DE LA CAMARA)", directores_texto)
        
        metadatos["directores"] = {}
        for name, entidad in directores:
            if "SENADO" in entidad:
                metadatos["directores"]["senado"] = name.strip()
            else:
                metadatos["directores"]["camara"] = name.strip()

    # --- 3. Edición de n páginas --- (Regex)
    patterns_pages = r"(?i)EDICI[ÓO]N\s+DE\s+(\d+)\s+P[ÁA]GINAS"
    match_pages = re.search(patterns_pages, texts_pdf)
    if match_pages:
        metadatos["num_paginas"] = int(match_pages.group(1))

    # --- 4. Año (Números Romanos) --- (Regex)
    pattern_anio = r"AÑO\s+(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))"
    match_anio = re.search(pattern_anio, texts_pdf)
    if match_anio:
        romano = match_anio.group(1)
        metadatos["anio_romano"] = romano
        metadatos["anio"] = romano_a_entero(romano)

    # --- 5. Nombre Principal ---
    pattern_name = r"(?i)GACETA\s+DEL\s+CONGRESO"
    if re.search(pattern_name, texts_pdf):
        metadatos["nombre"] = "Gaceta del Congreso"

    # --- 6. Subtítulo ---
    pattern_subtitulo = r"(?i)GACETA\s+DEL\s+CONGRESO\s*\n*(.*?)\s*(?:AÑO|DIRECTORES)"
    match_subtitulo = re.search(pattern_subtitulo, texts_pdf)
    if match_subtitulo:
        metadatos["subtitulo"] = match_subtitulo.group(1).strip()
        
    # --- 7. Artículo, Ley ---
    pattern_articule =  r"(?i)ART[ÍI]CULO\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*LEY\s+\d+(?:\s*(?:[º°ª])|\s*[Nn][°º])[\s.,:;]*DE[\s.,:;]*\d+"
    if re.search(pattern_articule, texts_pdf):
        metadatos["articulo_ley"] = True

    # --- 8. ISSN --- (Regex)
    pattern_issn = r"(?i)ISSN\s+(\d{4}-\d{3}[\dX])"
    match_issn = re.search(pattern_issn, texts_pdf)
    if match_issn:
        metadatos["issn"] = match_issn.group(1)

    # --- 9. Rama Legislativa --- (Regex)
    pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+P[ÚU]BLICO"
    if re.search(pattern_branch, texts_pdf):
        metadatos["rama"] = "Rama Legislativa del Poder Público"

    # --- 10. Cámara o Senado ---
    pattern_entity = r"(?i)(C[ÁA]MARA\s+DE\s+REPRESENTANTES|SENADO\s+DE\s+LA\s+REP[ÚU]BLICA)"
    match_entity = re.search(pattern_entity, texts_pdf, re.IGNORECASE)
    if match_entity:
        metadatos["camara"] = match_entity.group(1)

   # --- 11. Tipo de Documento ---
    pattern_type = r"^(Ponencia|Acta|Proyecto\s+de\s+Ley|Informe|Resolución|Concepto|Proposición|Constancia|Objeciones|Textos|Pliego)(.*)"
    for sent in doc.sents:
        match_type = re.search(pattern_type, sent.text, re.IGNORECASE)
        if match_type:
            metadatos["tipo_documento"] = match_type.group(1).strip()
            metadatos["descripcion"] = match_type.group(2).strip()
            break

    # --- Presentation ---
    pattern_presentation = r"(?i)PRESENTACI[ÓO]N\s*:\s*(.*?)\s*(?:DIRECTORES|RAMA|EDICI[ÓO]N)"
    match_presentation = re.search(pattern_presentation, texts_pdf)
    if match_presentation:
        metadatos["presentacion"] = match_presentation.group(1).strip()

    return metadatos

def romano_a_entero(romano):
    """Convierte un número romano a entero."""
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

def leer_pdf(ruta_pdf):
    """Lee un archivo PDF y devuelve su contenido en texto."""
    documento = fitz.open(ruta_pdf)
    texto = ""
    for pagina in documento:
        texto += pagina.get_text()
    return texto

def procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida):
    """Procesa todos los archivos PDF en una carpeta y guarda los metadatos en archivos JSON separados."""
    for archivo in os.listdir(carpeta_pdf):
        if archivo.endswith(".pdf"):
            ruta_pdf = os.path.join(carpeta_pdf, archivo)
            texts_pdf = leer_pdf(ruta_pdf)
            metadatos = extraer_metadatos_gaceta_spacy(texts_pdf)
            nombre_json = os.path.splitext(archivo)[0] + ".json"
            ruta_json = os.path.join(carpeta_salida, nombre_json)
            with open(ruta_json, "w", encoding="utf-8") as f:
                json.dump(metadatos, f, ensure_ascii=False, indent=4)
            print(f"Metadatos guardados en: {ruta_json}")

# Ruta de la carpeta que contiene los archivos PDF
carpeta_pdf = "c:/Users/Jorge/OneDrive/Documents/proyect/document"

# Ruta de la carpeta donde se guardarán los archivos JSON
carpeta_salida = "c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output"
os.makedirs(carpeta_salida, exist_ok=True)

# Procesar todos los archivos PDF en la carpeta
procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida)

Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_05.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1154.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1335.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1406.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1515.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_155.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1596.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_171.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output\gaceta_1775.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/do